In [1]:
import numpy as np
import cv2 as cv
import sys
import getopt
import glob
from multiprocessing.dummy import Pool as ThreadPool
from common import splitfn
import os

# global settings
square_size = 35.15
threads_num = 8
subpix_windows_size = (5, 5)
pattern_size = (6, 9)

term = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_COUNT, 1000, 1e-6)

dataset_dir = './mav_datasets/calibration_frontcam/img/'
img_dir = './mav_datasets/calibration_frontcam/img_selected_0/'
debug_dir = './mav_datasets/calibration_frontcam/debug/'
undistort_dir = './mav_datasets/calibration_frontcam/undistort/'


In [2]:
# Arrays to store object points and image points from all the images.

obj_points = []
img_points = []
images = glob.glob(img_dir + '*.jpg')
os.system('rm -rf ' + debug_dir)
os.system('mkdir ' + debug_dir)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0) * square_size
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

# find chessboard function
def processImg(fn):
    img = cv.imread(fn, cv.IMREAD_GRAYSCALE)
    found, corners = cv.findChessboardCorners(img, pattern_size)
    if found:
        corners_subpix = cv.cornerSubPix(img, corners, subpix_windows_size, (-1, -1), term)

        if debug_dir:
            vis = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
            cv.drawChessboardCorners(vis, pattern_size, corners_subpix, found)
            _path, name, _ext = splitfn(fn)
            outfile = os.path.join(debug_dir, name + '.jpg')
            cv.imwrite(outfile, vis)

        return (corners_subpix.reshape(-1, 2), pattern_points)
        
    if not found:
        return None

if threads_num <= 1:
    chessboards = [processImg(fn) for fn in images]
else:
    pool = ThreadPool(threads_num)
    chessboards = pool.map(processImg, images)

detected = [x for x in chessboards if x is not None]
for (corners, pattern_points) in detected:
    img_points.append(corners)
    obj_points.append(pattern_points)

In [3]:
# calculate camera distortion with all available images
print("calibrating, will take a long time if there are too many points")
h, w = cv.imread(images[0], cv.IMREAD_GRAYSCALE).shape[:2]
rms, camera_matrix, distortion, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, (w, h), None, None)

print("\nRMS:", rms)
print("camera matrix:\n", camera_matrix)
print("distortion coefficients: ", distortion.ravel())

calibrating, will take a long time if there are too many points

RMS: 0.2855858011133064
camera matrix:
 [[323.30439544   0.         264.1811044 ]
 [  0.         323.73762591 213.45702275]
 [  0.           0.           1.        ]]
distortion coefficients:  [-3.42172094e-01  1.42618962e-01  6.17475602e-05  1.79552857e-04
 -3.15633611e-02]


In [4]:
# use opencv undistort
os.system('rm -rf ' + undistort_dir)
os.system('mkdir ' + undistort_dir)

print(camera_matrix)
print(distortion)

images = glob.glob(dataset_dir + '*.jpg')
for fn in images if dataset_dir else []:
    _path, name, _ext = splitfn(fn)
    img_found = os.path.join(dataset_dir, name + '.jpg')
    outfile = os.path.join(undistort_dir, name + '.jpg')

    img = cv.imread(img_found)
    if img is None:
        continue

    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(camera_matrix, distortion, (w, h), 1, (w, h))
    dst = cv.undistort(img, camera_matrix, distortion, None, newcameramtx)

    # crop and save the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]

    cv.imwrite(outfile, dst)

[[323.30439544   0.         264.1811044 ]
 [  0.         323.73762591 213.45702275]
 [  0.           0.           1.        ]]
[[-3.42172094e-01  1.42618962e-01  6.17475602e-05  1.79552857e-04
  -3.15633611e-02]]
